# **Clustering Ministry Teams**

**Problem:** The Office of Ministry and Evangelism has 18 ministry teams as of November 2024. We are on pace to add 2-3 ministry teams per year. The Christian Service Council has Ministry Team Coordinators (MTC) that are each assigned 5-6 ministry teams to oversee and advise. However, in the past, these teams have been divided up sort of randomly which makes it hard for MTCs to provide targed assistance and care for the Ministry Team Directors. Thus, we seek to find a more systematic way to group similar ministry teams together in order to help MTCs focus their attention and care.

**Approach:** Each month, MTCs conduct one-on-one meetings with the directors of the ministry teams that they oversee. In November 2024, we instructed each MTC to collect specific data about the operation of each ministry team so that we could a) gain a better understanding of how each ministry team is operating and b) collect data in order to cluster similar ministry teams together. We will employ various clustering algorithms to get an objective opinion on which ministry teams are the most similar, evaluate the importance of each factor in order to understand what makes certain ministry teams similar to one another, and finally use the results as insight into how we can advise future CSC cabinets on how they should group ministry teams to provide more targeted care.

**Data:**


In [4]:
import pandas as pd   
import numpy as np    
import matplotlib.pyplot as plt   
import seaborn as sns   
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn import metrics


In [6]:
import os
data_dir = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/life things/data science/data/'
os.chdir(data_dir)
df = pd.read_csv('clustering ministry teams - Sheet3.csv')
df


,name,affiliation,advisor_name,advisor_involvement,event_type,cab_meetings,resources,budget,charter_alignment,target_audience
0,Engage Chicago,wheaton ministry partnering with affiliated mi...,Vince Bacote,0,somewhat weekly,consistent,3,okay,needs revision,wheaton students
1,GPP,wheaton ministry,Kathryn Manerio (OME document)\nDr. Romen (uno...,5,unknown,unknown,3,very good,needs revision,wheaton students
2,JJM,wheaton ministry partnering with affiliated mi...,Derk Polansky - Affiliated Partner,3,weekly events + other events,Consistent,3,okay,well aligned - not recently revised,off campus
3,MIF,wheaton ministry,n/a,0,seasonal,Consistent,3,good,no charter,whole campus
4,Samaritan's Purse,affiliated ministry,Matt Cain - Affiliated Partner,2,seasonal,Consistent,3,good,no charter,whole campus
5,Abide,wheaton ministry,Delaney Getz,5,weekly events + other events,Consistent,unknown,good,well aligned,wheaton women
6,Decision Point,affiliated ministry,Ryan Smith - Affiliated Partner,5,weekly events + other events,Consistent,3,good,no charter,off campus
7,Tutoring,wheaton ministry,Sara Vroom Fick,5,weekly,Consistent,unknown,not good,well aligned - recently revised,wheaton freshmen
8,VFL,wheaton ministry,Bryan McGraw,4,weekly events + other events,Consistent,3,good,well aligned - recently revised,unknown
9,Zoe's Feet,wheaton ministry,Dyanne Martin,5,seasonal,unknown,3,very good,well aligned - recently revised,whole campus


In [8]:
student_info_form = pd.read_excel('ME Student Info 24-25.xlsx', sheet_name='ME Student Info 24-25')
student_info_form

,Submission Date,First Name,Last Name,Sex,Race/Ethnicity,Wheaton Student Email,Phone Number,CPO Number,Where are you living Fall 2024?,Room Number,...,What is your Wheaton Username?,Are you certified to drive Wheaton vehicles?,Please Upload your Headshot,Please list any food allergies or dietary restrictions,Favorite Snack/Treat,Would you be interested in helping to lead worship for Ministry & Evangelism worship times during Student Leadership Training or throughout the year?,"What instrument(s) do you play? (voice, guitar, drums, etc.)",What are you up to this summer?,How can we be praying for you?,Submission ID
0,2024-08-07 12:02:26,Maddy,Buonincontro,Female,White or European American,maddy.buonincontro@my.wheaton.edu,(608) 4367399,465,Smith,NaN,...,maddybuonincontro,No,https://www.jotform.com/uploads/WheatonCollege...,NaN,Cheez its,NaN,NaN,Working at a summer camp in Minnesota !,NaN,5988593463179337573
1,2024-08-05 19:24:51,Sophia,Michel,Female,White or European American,sophia.michel@my.wheaton.edu,(773) 8120405,NaN,Smith,NaN,...,sophiamichel,No,https://www.jotform.com/uploads/WheatonCollege...,NaN,NaN,NaN,NaN,NaN,NaN,5987130916653831836
2,2024-08-05 14:22:08,Frances (Frankie),Kendziera,Female,White or European American,frankie.kendziera@my.wheaton.edu,(616) 4064461,1593,Fischer,W316,...,frankiekendziera,No,https://www.jotform.com/uploads/WheatonCollege...,NaN,Almond Joy,Yes,"Voice, piano",I have loved spending time and having conversa...,I would appreciate prayer for the upcoming sch...,5986949285915551025
3,2024-07-28 15:54:35,Lucy,Anderson,Female,White or European American,lucy.anderson@my.wheaton.edu,(630) 9627968,108,Smith,NaN,...,lucyanderson,No,https://www.jotform.com/uploads/WheatonCollege...,NaN,peanut m&ms,No,NaN,"I have been nannying a ton, as well as spendin...",I would love prayer for ongoing contentment in...,5980092759261051149
4,2024-07-17 09:08:53,Anna,Ziel,Female,White or European American,annaglow17@gmail.com,(719) 2441510,3286,Terrace,NaN,...,annaziel,No,https://www.jotform.com/uploads/WheatonCollege...,NaN,Anything chocolate,No,NaN,Wheaton in England!,Finishing the summer well,5970345332557429948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,2024-11-12 15:17:59,Maura,Wilkerson,Female,White or European American,maura.wilkerson@my.wheaton.edu,(405) 3153385,3053,Williston,311,...,maurawilkerson,No,https://www.jotform.com/uploads/WheatonCollege...,NaN,dark chocolate covered pretzels,No,NaN,NaN,"It's been a very busy semester, so i wold love...",6072554792172841047
102,2024-11-12 15:35:56,Trevor,Aston,Male,White or European American,trevor.aston@my.wheaton.edu,(419) 4192616798,54,Campus House,NaN,...,trevoraston,No,https://www.jotform.com/uploads/WheatonCollege...,NaN,Oreos,No,NaN,NaN,To work hard in school throughout the rest of ...,6072565566177076997
103,2024-11-12 18:42:42,Simon,Kutnow,Male,White or European American,simon.kutnow@my.wheaton.edu,(614) 9802829,1651,Terrace,NaN,...,simonkutnow,Yes,https://www.jotform.com/uploads/WheatonCollege...,lactose intolerant,flavored pretzel bites,No,NaN,NaN,Time management and presence in the moment,6072677628171512620
104,2024-11-13 12:09:09,Andrew,Moreno,Male,White or European American,drew.moreno@my.wheaton.edu,(720) 9571477,NaN,McManis Evans,NaN,...,drewmoreno,No,https://www.jotform.com/uploads/WheatonCollege...,NaN,Cheez-its,No,NaN,NaN,NaN,6073305490179541830


In [9]:
student_info_form.columns

Index(['Submission Date', 'First Name', 'Last Name', 'Sex', 'Race/Ethnicity',
       'Wheaton Student Email', 'Phone Number', 'CPO Number',
       'Where are you living Fall 2024?', 'Room Number',
       'Apartment Building Number', 'Apartment Number',
       'House Name and/or Address', 'Street Address', 'Street Address Line 2',
       'City', 'State / Province', 'Postal / Zip Code', 'Country',
       'Graduation Year', 'Birthdate', 'T-shirt Size', 'I am a...',
       'What ministry are you a part of?',
       'What is your position title within your ministry?',
       'Are you a part of an additional ministry within Ministry & Evangelism?',
       'What ministry are you a part of? 2',
       'What is your position title within the ministry listed above?',
       'What date do you plan to arrive back to campus for Student Leadership Training in August?',
       'Please share why you are unable to return at your given time for your group. Please reach out to Jared and Delaney to come u

In [15]:
sif_mt_names = student_info_form['What ministry are you a part of?'].sort_values().unique().tolist()
sif_mt_names

['A Rocha',
 'Abide',
 'Christian Service Council',
 'Decision Point',
 'Engage Chicago',
 'Garden & Prairie Project',
 'Honduras Project',
 'IJM',
 'Juvenile Justice Ministry',
 'Missions in Focus (MIF)',
 'OHANA',
 'Pre-Health Ministry Team',
 "Samaritan's Purse",
 'Skivangelism',
 'Student Missionary Partners (SMP)',
 'Tutoring Ministry Team',
 'Unbroken',
 'Voice for Life',
 'World Christian Fellowship (WCF)',
 "Zoe's Feet"]

In [16]:
df_mt_names = df['name'].sort_values(ascending=True).unique().tolist()
df_mt_names

['A Rocha',
 'Abide',
 'Decision Point',
 'Engage Chicago',
 'GPP',
 'IJM',
 'JJM',
 'MIF',
 'OHANA',
 'Pre-Health',
 "Samaritan's Purse",
 'Skivangelism',
 'Tutoring',
 'Unbroken',
 'VFL',
 "Zoe's Feet"]

In [32]:
# Assuming 'student_info_form' is your DataFrame
student_info_form_mt = student_info_form[~student_info_form['What ministry are you a part of?'].isin(['Honduras Project', 'Student Missionary Partners (SMP)', 'World Christian Fellowship (WCF)', 'Christian Service Council'])]
sif_mt_names = student_info_form_mt['What ministry are you a part of?'].sort_values().unique().tolist()

In [33]:
sif_mt_names

['A Rocha',
 'Abide',
 'Decision Point',
 'Engage Chicago',
 'Garden & Prairie Project',
 'IJM',
 'Juvenile Justice Ministry',
 'Missions in Focus (MIF)',
 'OHANA',
 'Pre-Health Ministry Team',
 "Samaritan's Purse",
 'Skivangelism',
 'Tutoring Ministry Team',
 'Unbroken',
 'Voice for Life',
 "Zoe's Feet"]

In [34]:
df_mt_names

['A Rocha',
 'Abide',
 'Decision Point',
 'Engage Chicago',
 'GPP',
 'IJM',
 'JJM',
 'MIF',
 'OHANA',
 'Pre-Health',
 "Samaritan's Purse",
 'Skivangelism',
 'Tutoring',
 'Unbroken',
 'VFL',
 "Zoe's Feet"]

In [36]:
sif_to_df_dict = dict(zip(sif_mt_names, df_mt_names))
sif_to_df_dict

{'A Rocha': 'A Rocha',
 'Abide': 'Abide',
 'Decision Point': 'Decision Point',
 'Engage Chicago': 'Engage Chicago',
 'Garden & Prairie Project': 'GPP',
 'IJM': 'IJM',
 'Juvenile Justice Ministry': 'JJM',
 'Missions in Focus (MIF)': 'MIF',
 'OHANA': 'OHANA',
 'Pre-Health Ministry Team': 'Pre-Health',
 "Samaritan's Purse": "Samaritan's Purse",
 'Skivangelism': 'Skivangelism',
 'Tutoring Ministry Team': 'Tutoring',
 'Unbroken': 'Unbroken',
 'Voice for Life': 'VFL',
 "Zoe's Feet": "Zoe's Feet"}

In [39]:
student_info_form_mt['What ministry are you a part of?'] = student_info_form_mt['What ministry are you a part of?'].map(sif_to_df_dict)

/var/folders/99/6591xgpn4sq1ypfgnrzy_7n80000gn/T/ipykernel_17558/2057012429.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student_info_form_mt['What ministry are you a part of?'] = student_info_form_mt['What ministry are you a part of?'].map(sif_to_df_dict)


In [43]:
df

,name,affiliation,advisor_name,advisor_involvement,event_type,cab_meetings,resources,budget,charter_alignment,target_audience
0,Engage Chicago,wheaton ministry partnering with affiliated mi...,Vince Bacote,0,somewhat weekly,consistent,3,okay,needs revision,wheaton students
1,GPP,wheaton ministry,Kathryn Manerio (OME document)\nDr. Romen (uno...,5,unknown,unknown,3,very good,needs revision,wheaton students
2,JJM,wheaton ministry partnering with affiliated mi...,Derk Polansky - Affiliated Partner,3,weekly events + other events,Consistent,3,okay,well aligned - not recently revised,off campus
3,MIF,wheaton ministry,n/a,0,seasonal,Consistent,3,good,no charter,whole campus
4,Samaritan's Purse,affiliated ministry,Matt Cain - Affiliated Partner,2,seasonal,Consistent,3,good,no charter,whole campus
5,Abide,wheaton ministry,Delaney Getz,5,weekly events + other events,Consistent,unknown,good,well aligned,wheaton women
6,Decision Point,affiliated ministry,Ryan Smith - Affiliated Partner,5,weekly events + other events,Consistent,3,good,no charter,off campus
7,Tutoring,wheaton ministry,Sara Vroom Fick,5,weekly,Consistent,unknown,not good,well aligned - recently revised,wheaton freshmen
8,VFL,wheaton ministry,Bryan McGraw,4,weekly events + other events,Consistent,3,good,well aligned - recently revised,unknown
9,Zoe's Feet,wheaton ministry,Dyanne Martin,5,seasonal,unknown,3,very good,well aligned - recently revised,whole campus


# ***go ahead and cluster without much extra info***

In [44]:
kmeans = KMeans(n_clusters = 3, random_state=11)
kmeans.fit(df)

df['Cluster'] = kmeans.labels_

df

ValueError: could not convert string to float: 'Engage Chicago'

In [45]:
df

,name,affiliation,advisor_name,advisor_involvement,event_type,cab_meetings,resources,budget,charter_alignment,target_audience
0,Engage Chicago,wheaton ministry partnering with affiliated mi...,Vince Bacote,0,somewhat weekly,consistent,3,okay,needs revision,wheaton students
1,GPP,wheaton ministry,Kathryn Manerio (OME document)\nDr. Romen (uno...,5,unknown,unknown,3,very good,needs revision,wheaton students
2,JJM,wheaton ministry partnering with affiliated mi...,Derk Polansky - Affiliated Partner,3,weekly events + other events,Consistent,3,okay,well aligned - not recently revised,off campus
3,MIF,wheaton ministry,n/a,0,seasonal,Consistent,3,good,no charter,whole campus
4,Samaritan's Purse,affiliated ministry,Matt Cain - Affiliated Partner,2,seasonal,Consistent,3,good,no charter,whole campus
5,Abide,wheaton ministry,Delaney Getz,5,weekly events + other events,Consistent,unknown,good,well aligned,wheaton women
6,Decision Point,affiliated ministry,Ryan Smith - Affiliated Partner,5,weekly events + other events,Consistent,3,good,no charter,off campus
7,Tutoring,wheaton ministry,Sara Vroom Fick,5,weekly,Consistent,unknown,not good,well aligned - recently revised,wheaton freshmen
8,VFL,wheaton ministry,Bryan McGraw,4,weekly events + other events,Consistent,3,good,well aligned - recently revised,unknown
9,Zoe's Feet,wheaton ministry,Dyanne Martin,5,seasonal,unknown,3,very good,well aligned - recently revised,whole campus


In [54]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_columns = ['name', 'affiliation', 'advisor_name', 'event_type', 'cab_meetings', 'budget','resources', 'charter_alignment', 'target_audience']
df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

scaler = StandardScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df_encoded), columns=df_encoded.columns)

kmeans = KMeans(n_clusters=4, random_state=11)
kmeans.fit(df_normalized)

df['Cluster'] = kmeans.labels_

In [55]:
df

,name,affiliation,advisor_name,advisor_involvement,event_type,cab_meetings,resources,budget,charter_alignment,target_audience,Cluster
0,Engage Chicago,wheaton ministry partnering with affiliated mi...,Vince Bacote,0,somewhat weekly,consistent,3,okay,needs revision,wheaton students,0
1,GPP,wheaton ministry,Kathryn Manerio (OME document)\nDr. Romen (uno...,5,unknown,unknown,3,very good,needs revision,wheaton students,3
2,JJM,wheaton ministry partnering with affiliated mi...,Derk Polansky - Affiliated Partner,3,weekly events + other events,Consistent,3,okay,well aligned - not recently revised,off campus,0
3,MIF,wheaton ministry,n/a,0,seasonal,Consistent,3,good,no charter,whole campus,1
4,Samaritan's Purse,affiliated ministry,Matt Cain - Affiliated Partner,2,seasonal,Consistent,3,good,no charter,whole campus,1
5,Abide,wheaton ministry,Delaney Getz,5,weekly events + other events,Consistent,unknown,good,well aligned,wheaton women,3
6,Decision Point,affiliated ministry,Ryan Smith - Affiliated Partner,5,weekly events + other events,Consistent,3,good,no charter,off campus,3
7,Tutoring,wheaton ministry,Sara Vroom Fick,5,weekly,Consistent,unknown,not good,well aligned - recently revised,wheaton freshmen,2
8,VFL,wheaton ministry,Bryan McGraw,4,weekly events + other events,Consistent,3,good,well aligned - recently revised,unknown,3
9,Zoe's Feet,wheaton ministry,Dyanne Martin,5,seasonal,unknown,3,very good,well aligned - recently revised,whole campus,3


In [56]:
df.query('Cluster == 1')

,name,affiliation,advisor_name,advisor_involvement,event_type,cab_meetings,resources,budget,charter_alignment,target_audience,Cluster
3,MIF,wheaton ministry,n/a,0,seasonal,Consistent,3,good,no charter,whole campus,1
4,Samaritan's Purse,affiliated ministry,Matt Cain - Affiliated Partner,2,seasonal,Consistent,3,good,no charter,whole campus,1
11,IJM,affiliated ministry,Stephen Bretsen,5,other,Consistent,3,good,well aligned,whole campus,1
12,OHANA,wheaton ministry,Paul Lee,3,monthly,Consistent,unknown,not good,well aligned,whole campus,1
14,Skivangelism,wheaton ministry,Steve Ivester,4,Seasonal\n2 events for fall; weekly\nin A quad...,Consistent,unknown,not good,well aligned,wheaton students,1
15,Unbroken,wheaton ministry,Russ Streff,0,Weekly,Consistent,unknown,okay,well aligned,wheaton men,1


In [57]:
df.query('Cluster == 2')

,name,affiliation,advisor_name,advisor_involvement,event_type,cab_meetings,resources,budget,charter_alignment,target_audience,Cluster
7,Tutoring,wheaton ministry,Sara Vroom Fick,5,weekly,Consistent,unknown,not good,well aligned - recently revised,wheaton freshmen,2


In [58]:
df.query('Cluster == 3')

,name,affiliation,advisor_name,advisor_involvement,event_type,cab_meetings,resources,budget,charter_alignment,target_audience,Cluster
1,GPP,wheaton ministry,Kathryn Manerio (OME document)\nDr. Romen (uno...,5,unknown,unknown,3,very good,needs revision,wheaton students,3
5,Abide,wheaton ministry,Delaney Getz,5,weekly events + other events,Consistent,unknown,good,well aligned,wheaton women,3
6,Decision Point,affiliated ministry,Ryan Smith - Affiliated Partner,5,weekly events + other events,Consistent,3,good,no charter,off campus,3
8,VFL,wheaton ministry,Bryan McGraw,4,weekly events + other events,Consistent,3,good,well aligned - recently revised,unknown,3
9,Zoe's Feet,wheaton ministry,Dyanne Martin,5,seasonal,unknown,3,very good,well aligned - recently revised,whole campus,3
10,A Rocha,affiliated ministry,Ben Low - Affiliated Partner,2,monthly,Consistent,unknown,unknown,well aligned,unknown,3


In [59]:
df.query('Cluster == 0')

,name,affiliation,advisor_name,advisor_involvement,event_type,cab_meetings,resources,budget,charter_alignment,target_audience,Cluster
0,Engage Chicago,wheaton ministry partnering with affiliated mi...,Vince Bacote,0,somewhat weekly,consistent,3,okay,needs revision,wheaton students,0
2,JJM,wheaton ministry partnering with affiliated mi...,Derk Polansky - Affiliated Partner,3,weekly events + other events,Consistent,3,okay,well aligned - not recently revised,off campus,0
13,Pre-Health,wheaton ministry partnering with affiliated mi...,Candice Eisenahuer,3,Monthly,Consistent,2,good,needs revision,wheaton students,0
